# Create WR database for logistic regression for D3 visualization

In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import json

pd.options.display.max_rows = 999
pd.options.display.max_columns = 110

In [ ]:
file_path = 'collegefb/combine_update.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['year','player','pos','college','height','weight','forty','vertical','bench','broad','threecone','shuttle','drafted','url']

df = pd.DataFrame(data,columns=labels)

df["height"]= (df.height.str.split('-').str[0]).astype(int)*12 + (df.height.str.split('-').str[1]).astype(int)
df["firstround"]= ((df.drafted.str.split('/').str[1].str.strip())=='1st').astype(int)
df = df.convert_objects(convert_numeric=True)

df["round"]= df.drafted.str.split('/').str[1].str.strip().str[:1]
df["pick"]=df.drafted.str.split('/').str[2].str.strip().str[:-7]
df['round'].fillna(0, inplace=True)
df['pick'].fillna(0, inplace=True)
df = df.convert_objects(convert_numeric=True)

In [ ]:
file_path = 'collegefb/draft_update2.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['year','round','pick','team','player','pos','age','college','url']

df_draft = pd.DataFrame(data,columns=labels)
df_draft = df_draft.convert_objects(convert_numeric=True)
df_draft = df_draft[['year','round','pick','player','pos','college','url']]

In [ ]:
file_path = 'collegefb/college_update2.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
#labels = ['completions','attempts','pass_yards','pass_tds','rec_yards','rec_td','receptions', 'rush_att', 'rush_yds', 'rush_td', 'solo_tackes', 'tackles', 'loss_tackles', 'ast_tackles', 'fum_forced', 'sacks', 'int', 'pd']
labels = ['player_name', 'receptions', 'rec_yards', 'rec_td', 'rush_att', 'rush_yds', 'rush_td', 'attempts', 'completions', 'pass_ints', 'pass_tds', 'pass_yards','games', 'solo_tackles', 'ast_tackles', 'tackles', 'loss_tackles', 'sacks', 'int', 'int_tds', 'int_yds', 'pd', 'fum_rec', 'fum_forced', 'fum_tds', 'fum_yds','url']

df_college = pd.DataFrame(data,columns=labels)
df_college = df_college.convert_objects(convert_numeric=True)

In [ ]:
df_college_norm=pd.DataFrame(df_college['url'])

cols = ['receptions', 'rec_yards', 'rec_td', 'rush_att', 'rush_yds', 'rush_td', 'attempts', 'completions', 'pass_ints', 'pass_tds', 'pass_yards','games', 'solo_tackles', 'ast_tackles', 'tackles', 'loss_tackles', 'sacks', 'int', 'int_tds', 'int_yds', 'pd', 'fum_rec', 'fum_forced', 'fum_tds', 'fum_yds']

for col in cols:
    df_college_norm[col] = df_college[col]/(df_college[col].max())

df_college = df_college.drop('player_name',1)

# Rankings

In [ ]:
file_path = 'collegefb/cfb_rank.json'
with open(file_path) as json_data:
    d = json.load(json_data)

    data = []
for row in d:
    for k in row:
        data.append(row[k])
labels = ['poll','rank','college']

df_rank = pd.DataFrame(data,columns=labels)

In [ ]:
df_rank['year']=df_rank.poll.str.split(' ').str[0].astype(int)+1
df_rank['college_r']=1

# Combine datasets

In [ ]:
temp_df = df.merge(df_rank,how='left',on=['college','year'])
complete_df = temp_df.merge(df_college,how='left',on=['url'])
complete_df.fillna(0, inplace=True)
wr_df = complete_df[complete_df['pos']=='WR']

In [ ]:
wr_df_ab=wr_df[['height','weight','firstround','college_r','receptions','rec_yards','rec_td']]
wr_df_ab.to_pickle('wr_data.pkl')